#### Script Silver to Gold Producció.
- Executa les comandes mínimes però necessaries per realitzar el procès més optim possible.
- Per veure com s'ha arribat a desenvolupar aquest df i que fa cada pas mirar el Notebook de documentació.

#### Importem les llibreries necessàries

In [0]:
import pandas as pd
from pyspark.sql.functions import col, to_json

#### Configuració connexió spark.

In [0]:
token = 'sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-06-03T02:34:04Z&st=2024-04-09T18:34:04Z&spr=https&sig=dplYYYNSRC%2FjDR89WMxcsA6SMo%2BHFYrQ5BqdhRUKBMs%3D'
storage_account = 'projecteiabd'
container = 'silver'

spark.conf.set("fs.azure.account.auth.type.{0}.dfs.core.windows.net".format(storage_account), "SAS")
spark.conf.set("fs.azure.sas.token.provider.type.{0}.dfs.core.windows.net".format(storage_account), "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set("fs.azure.sas.fixed.token.{0}.dfs.core.windows.net".format(storage_account), token)


#### Declaració de funcions.

In [0]:
# Funció per llegir tots els fitxer ".parquet" de la partició que li assignem.
import pandas as pd

def read_all_silver(particio, cotainer, storage_account):
     docs = []

     # Li diguem la carpeta on tenim les particons. I ja els ha llegeix totes automàticament, no fa falta la funció recursiva.
     path = 'abfs://{0}@{1}.dfs.core.windows.net/{2}'.format(container, storage_account,particio)
     
     df = spark.read.parquet(path)
     
     return df


In [0]:
# Funció per llegir tiquets d'un client concret
def read_ticket_client(id_client, cotainer, storage_account):
     docs = []

     # Li diguem la carpeta on tenim les particons. I ja els ha llegeix totes automàticament, no fa falta la funció recursiva.
     path = 'abfs://{0}@{1}.dfs.core.windows.net/clients/id_client={2}'.format(container, storage_account, id_client)
     
     df = spark.read.parquet(path)
     
     return df

In [0]:
# Funció per llegir tiquets d'un día concret
def read_ticket_data(data, cotainer, storage_account):
     docs = []

     # Li diguem la carpeta on tenim les particons. I ja els ha llegeix totes automàticament, no fa falta la funció recursiva.
     path = 'abfs://{0}@{1}.dfs.core.windows.net/dies/data={2}'.format(container, storage_account, data)
     
     df = spark.read.parquet(path)
     
     return df


#### Tractament de dades per penjar a SQL SERVER

In [0]:
# En aquest cas per passar a la capa gold(SQL) agafem totes les dades de una partició sense importar quina sigui ja que de aquesta forma penjarem totes les dades a la capa Gold. 
# Que es el que ens interessa poder mostrar estadístiques i gràfiques de totes les dades en aquest cas.

# Llegim tots els parquets, amb la funció per llegir-ho tot de totes les particións.
df_parquet = read_all_silver('clients', container, storage_account)

# Convertim el resultat en un df de pandas.
df_parquet = df_parquet.toPandas() 

In [0]:
product_data = []
tiquets_data = []

for index, row in df_parquet.iterrows():
    tiquets_data.append({
        'id_tiquet': row['id_tiquet'],
        'id_client': row['id_client'],
        'total_amb_IVA': row['total_amb_IVA'],
        'data_scanner_tiquet': row['data_scanner_tiquet'],
        'data_tiquet': row['data'],
        'ciutat': row['ciutat'],
    })

    for producte in row['products']:
        product_data.append({
            'descripcio': producte['descripcio'],
            'quantitat': producte['quantitat'],
            'preu unitari': producte['preu_unitari'],
            'import': producte['import'],
            'unitat': producte['unitat'],
            'id_tiquet': row['id_tiquet']
        })

# Converitm l'array de objectes amb un dataframe.
df_productes_bi = pd.DataFrame(product_data)
df_tiquets_bi = pd.DataFrame(tiquets_data)


#### Penjem les dades al servidor Azure SQL Server

In [0]:
from pyspark.sql.functions import col, to_json


# Cadena de connexió JDBC (llibreria de connexió)
jdbc_url = "jdbc:sqlserver://sql-otorrent-etl.database.windows.net:1433;database=sqldb-otorrent-etl;"

# Propietats de connexió (Dades que utiltizem per realitzar les connexións)
connection_properties = {
    "user": "oriol@sql-otorrent-etl",
    "password": "41598051KKk",
    "encrypt": "true",
    "trustServerCertificate": "false",
    "hostNameInCertificate": "*.database.windows.net",
    "loginTimeout": "30"
}

# Panjem els dfs al sql server.
spark.createDataFrame(df_tiquets_bi).write.jdbc(url=jdbc_url, table="dbo.tiquets", mode="overwrite", properties=connection_properties)
spark.createDataFrame(df_productes_bi).write.jdbc(url=jdbc_url, table="dbo.lin_prodcutes", mode="overwrite", properties=connection_properties)